In [20]:
import numpy as np
import utils
from utils import loadmat
from extractDigitFeatures import extractDigitFeatures
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, Conv3D, Flatten, MaxPool2D, AveragePooling2D, BatchNormalization
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.python.keras import metrics
from keras import regularizers
from keras.constraints import unit_norm
from sklearn.model_selection import StratifiedKFold
from keras.callbacks import EarlyStopping

In [21]:
def get_random_data(data1, data2, low, high, max_samples=100):
    N, H1, W1, C1 = data1.shape
    #_, N1 = data2.shape
    suff_data1 = np.zeros((max_samples, H1, W1, C1))
    suff_data2 = np.zeros((max_samples,))
    shuffles = np.random.randint(low, high+1, max_samples)
    for idx in range(shuffles.shape[0]):
        suff_data1[idx] = data1[idx, :, :, :]
        suff_data2[idx] = data2[idx]
    return suff_data1, suff_data2

def reformat(x):
    new_x = np.zeros((x.shape[2], x.shape[0], x.shape[1]))
    for i in range(x.shape[2]):
        new_x[i, :] = x[:, :, i]
    return new_x[:, :, :, np.newaxis]

def return_datasets(filename):
    data = utils.loadmat('../data/{}'.format(filename))
    trainSet, testSet, valSet = 1, 2, 3
    
    x_train = reformat(data['x'][:, :, data['set']==trainSet ])
    y_train = (data['y'][data['set']==trainSet])
    x_val = reformat(data['x'][:, :, data['set']==valSet])
    y_val = (data['y'][data['set']==valSet])
    x_test = reformat(data['x'][:, :, data['set']==testSet])
    y_test = (data['y'][data['set']==testSet])
    
    x_train, y_train = get_random_data(x_train, y_train, 0, x_train.shape[0], x_train.shape[0])
    x_val, y_val = get_random_data(x_val, y_val, 0, x_val.shape[0], x_val.shape[0])
    x_test, y_test = get_random_data(x_test, y_test, 0, x_test.shape[0], x_test.shape[0])
    
    #print (x_train.shape, y_train.shape, x_val.shape, y_val.shape, x_test.shape, y_test.shape)
    

    #return (X, Y, x_test, y_test)
    return (x_train, y_train, x_val, y_val, x_test, y_test)

In [22]:
def extract_model(activation_func = 'relu'):
    model = Sequential()
    kernel_size=(5)
    stride_size = (2)

    model.add(Conv2D(32, kernel_size=kernel_size, strides=stride_size, activation=activation_func, padding='same', input_shape=(28, 28, 1)))
    model.add(MaxPool2D(2, 2))
    model.add(BatchNormalization())
    model.add(Conv2D(32, kernel_size=kernel_size, strides=stride_size, activation=activation_func, padding='same'))
    model.add(MaxPool2D(2, 2))
    model.add(BatchNormalization())

    model.add(Conv2D(64, kernel_size=kernel_size, strides=stride_size, activation=activation_func, padding='same'))
    #model.add(MaxPool2D(2, 2))
    model.add(BatchNormalization())
    
    model.add(Conv2D(64, kernel_size=kernel_size, strides=stride_size, activation=activation_func, padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(96, kernel_size=kernel_size, strides=stride_size, activation=activation_func, padding='same'))
    model.add(BatchNormalization())


    model.add(Flatten())
    model.add(Dense(256, activation=activation_func)) 
    model.add(Dense(10, activation='softmax'))
    
    return model

In [27]:
file = 'digits-jitter.mat'
x_train, y_train, x_val, y_val, x_test, y_test = return_datasets(file)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
best_model, best_accuracy = -1, -1

for train, test in kfold.split(x_train, y_train):
    curr_x_train, curr_y_train = x_train[train], to_categorical(y_train[train], num_classes=10)
    curr_x_val, curr_y_val = x_train[train], to_categorical(y_train[train], num_classes=10)
    
    model = extract_model()
    if file == 'digits-scaled.mat':
        model = model_extractor('selu')

    model.compile(loss="categorical_crossentropy", optimizer='Adam', validation_data=(curr_x_val, curr_y_val), metrics=["accuracy"])
    model.fit(curr_x_train, curr_y_train, epochs=MAX_EPOCHS, batch_size=MAX_BATCH_SIZE, verbose=1)
    accuracy = model.evaluate(x_val, to_categorical(y_val, num_classes=10), verbose=0)[1]
    print ("Accuracy = {}".format(accuracy))
    if accuracy > best_accuracy:
        best_accuracy, best_model = accuracy, model

print ("Test Set Accuracy for {} = {}".format(file, best_model.evaluate(x_test, to_categorical(y_test, num_classes=10), verbose=0)))
    
    

Train on 900 samples
Epoch 1/20
900/900 [==============================] - 2s 2ms/sample - loss: 2.2646 - accuracy: 0.2033
Epoch 2/20
900/900 [==============================] - 6s 7ms/sample - loss: 1.9465 - accuracy: 0.2800
Epoch 3/20
900/900 [==============================] - 1s 1ms/sample - loss: 1.7424 - accuracy: 0.3656
Epoch 4/20
900/900 [==============================] - 1s 829us/sample - loss: 1.4501 - accuracy: 0.4700
Epoch 5/20
900/900 [==============================] - 1s 825us/sample - loss: 1.2834 - accuracy: 0.5700
Epoch 6/20
900/900 [==============================] - 1s 987us/sample - loss: 1.0208 - accuracy: 0.6478
Epoch 7/20
900/900 [==============================] - 1s 866us/sample - loss: 0.9423 - accuracy: 0.6789
Epoch 8/20
900/900 [==============================] - 1s 918us/sample - loss: 0.8991 - accuracy: 0.6956
Epoch 9/20
900/900 [==============================] - 1s 941us/sample - loss: 0.7069 - accuracy: 0.7611
Epoch 10/20
900/900 [============================

900/900 [==============================] - 2s 2ms/sample - loss: 0.3262 - accuracy: 0.8911
Epoch 19/20
900/900 [==============================] - 2s 2ms/sample - loss: 0.2940 - accuracy: 0.9078
Epoch 20/20
900/900 [==============================] - 2s 2ms/sample - loss: 0.3192 - accuracy: 0.9011
Accuracy = 0.4659999907016754
Train on 900 samples
Epoch 1/20
900/900 [==============================] - 283s 314ms/sample - loss: 2.2795 - accuracy: 0.1867
Epoch 2/20
900/900 [==============================] - 2s 2ms/sample - loss: 1.9644 - accuracy: 0.3000
Epoch 3/20
900/900 [==============================] - 2s 2ms/sample - loss: 1.6517 - accuracy: 0.4178
Epoch 4/20
900/900 [==============================] - 2s 2ms/sample - loss: 1.4275 - accuracy: 0.5156
Epoch 5/20
900/900 [==============================] - 2s 2ms/sample - loss: 1.2084 - accuracy: 0.5767
Epoch 6/20
900/900 [==============================] - 2s 2ms/sample - loss: 1.0376 - accuracy: 0.6389
Epoch 7/20
900/900 [================

KeyboardInterrupt: 

In [25]:
# MAX_EPOCHS = 20
# MAX_BATCH_SIZE = 10

# files = ['digits-normal.mat', ] 
# for file in files:
#     x_train, y_train, x_val, y_val, x_test, y_test = return_datasets(file)

#     model = extract_model('relu')
#     model.compile(loss="categorical_crossentropy", optimizer='Adam', validation_data=(x_val, y_val), metrics=["accuracy"])
#     model.fit(x_train, y_train, epochs=MAX_EPOCHS, batch_size=MAX_BATCH_SIZE,  verbose=1)
#     accuracy = model.evaluate(x_test, y_test, verbose=0)[1]
#     print ("Accuracy = {}".format(accuracy))